# StaphPRECISE File Generator

This notebook generates all files for the Modulytics web page of Staph aureus, based on ICA of StaphPRECISE.

In [28]:
import sys
# be sure that this point to a clone of github.com/SBRG/ICA
sys.path.append('../../../../ica/')
from icaviz.plotting import *
DATA_DIR = 'data_files/'
GENE_DIR = '../annotation/gene_files/'
enrich = pd.read_csv(DATA_DIR + 'curated_enrichments.csv', index_col=0)
names = enrich['name'].tolist()
ica_data = load_data(X=DATA_DIR+'log_tpm.csv',
                     S=DATA_DIR+'M.csv',
                     A=DATA_DIR+'A.csv',
                     metadata='sample_metadata.csv',
                     annotation=GENE_DIR+'gene_info.csv',
                     trn=GENE_DIR+'TRN.csv',
                     cutoff=280, 
                     organism='saureus')
pd.set_option('display.max_rows', None)

In [29]:
not_in = 0
had_nan = 0
name = 0
for g in ica_data.S.index:
    if not(g in ica_data.num2name.keys()):
        ica_data.num2name[g] = g
        not_in += 1
    elif not(type(ica_data.num2name[g])==str):
        ica_data.num2name[g] = g
        had_nan += 1
    else:
        name += 1
print(name, had_nan, not_in)

1738 1030 52


In [30]:
# fill in biological replicates column
ica_data.metadata = ica_data.metadata.rename(columns = {'Biological Replicates':'rep_id'})

for name, group in ica_data.metadata.groupby(['project_id','condition_id']):
    num_reps = group.shape[0]
    #display(group)
    ica_data.metadata.loc[group.index, 'Biological Replicates'] = [int(num_reps)]*num_reps
ica_data.metadata['Biological Replicates'] = ica_data.metadata['Biological Replicates'].astype(int)

In [31]:
# get functions from py files
sys.path.append('../../../py')
from gene_table import *
from gene_histogram import *
from gene_scatter import *
from activity_bar import *
from regulon_venn import *
from regulon_scatter import *

# gene dashboards
from gene_activity_bar import *

import os

In [32]:
# read in other necessary annotation files
links = pd.read_csv('../annotation/gene_links.csv', index_col = 3)['link'].to_dict()
sample_meta = ica_data.metadata.reset_index()
gene_info = pd.read_csv(GENE_DIR+'gene_info.csv', index_col = 0)
trn = pd.read_csv(GENE_DIR+'TRN.csv', index_col = 0)

In [33]:
had_link = 0
no_link = 0
for g in ica_data.S.index:
    try: 
        links[g]
        had_link += 1
    except:
        links[g] = np.nan
        no_link += 1
print(had_link, no_link)

2506 314


In [34]:
def tf_with_links(k, row, links):
    tf_str = row.TF

    if not(type(tf_str) == str):
        return tf_str

    # get a list of transcription factors
    and_or = ''
    if '/' in tf_str:
        and_or = ' or '
        tfs = tf_str.split('/')
    elif '+' in tf_str:
        and_or = ' and '
        tfs = tf_str.split('+')
    else:
        tfs = [tf_str]

    # start building an html string
    tfs_html = []
    for tf in tfs:
        tf1 = tf[0].lower() + tf[1:]
        if tf1 in ica_data.name2num.keys():
            tf_gene = ica_data.name2num[tf1]
            link = links[tf_gene]
            if type(link)==str:# this tf has a link
                tf_ = '<a href="' + link + '" target="_blank">'+ tf + '</a>'
                tfs_html += [tf_]
            else: # this tf has no link
                tfs_html += [tf]
        # this tf isn't in the tf_links file
        else:
            tfs_html += [tf]
    res = and_or.join(tfs_html)
    return res

## Generate Files

In [35]:
# dataset_meta: Stores information for the header of the dataset page
num_genes = ica_data.X.shape[0]
num_samps = ica_data.X.shape[1]
num_conds = 0;
for n, g in ica_data.metadata.groupby(['condition_id', 'project_id']):
    num_conds += 1
num_ims = ica_data.S.shape[1]

dataset_meta = pd.Series({'Title':'<i>S. aureus</i> StaphPRECISE',
                          'Organism': '<i>Staphylococcus aureus</i> USA 300',
                          'Strain': 'TCH1516 and LAC',
                          'Publication':'<a href="https://doi.org/10.1101/2020.03.18.997296">Poudel, et al., 2020</a>',
                          'Number of Samples': num_samps,
                          'Number of Unique Conditions': num_conds,
                          'Number of Genes':num_genes,
                          'Number of iModulons': num_ims})
dataset_meta.to_csv('dataset_meta.csv')

/home/kevin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


In [36]:
cat_order = ['Virulence',
             'Carbon Source Utilization',
             'Amino Acid and Nucleotide Metabolism',
             'Energy Metabolism',
             'Metal Homeostasis',
             'Miscellaneous Metabolism',
             'Structural Components',
             'Stress Response',
             'Biological Enrichment',
             'Mobile Elements',
             'Genomic Strain Difference',
             'Uncharacterized']
cat_dict = {cat_order[i]:i for i in range(len(cat_order))}
im_table = enrich[['name', 'Regulator', 'Function', 'Category', 'n_genes', 'precision', 'recall']]
im_table = im_table.rename(columns={'name':'Name'})
im_table.index.name = 'k'
im_table['category_num'] = [cat_dict[im_table.Category[k]] for k in im_table.index]
im_table.to_csv('iM_table.csv')

In [37]:
num_ims = im_table.shape[0]-1
file = open('num_ims.txt', 'w')
file.write(str(num_ims))
file.close()

In [38]:
def make_directory(ica_data, k, row, links, sample_meta):
    # generate the plot files
    gene_table = gene_table_df(ica_data, k, row, links = links, operon_commas=False)
    gene_hist = gene_hist_df(ica_data, k, row)
    base_conds = ['USA300_TCH1516_U01-Set000_CAMHB_Control_1', 'USA300_TCH1516_U01-Set000_CAMHB_Control_2'] #TCH1516
    #base_conds = ['USA300_LAC_CAMHB_U01-Set001_Control_1', 'USA300_LAC_CAMHB_U01-Set001_Control_2']
    gene_scatter = gene_scatter_df(ica_data, k, base_conds, links)
    act_bar = activity_bar_df(ica_data, k, sample_meta)

    reg_venn = regulon_venn_df(ica_data, k, row)
    reg_scatter = regulon_scatter_df(ica_data, k, row)

    # generate a basic data df
    res = pd.Series(index=['name', 'TF', 'Regulator',
                           'Function', 'Category', 
                           'has_venn', 'scatter', 'has_meme'])
    res.loc['name'] = row.loc['name']
    res.loc['TF'] = row.TF
    res.loc['Regulator'] = tf_with_links(k, row, links)
    res.loc['Function'] = row.Function
    res.loc['Category'] = row.Category
    res.loc['has_venn'] = not(reg_venn is None)
    if reg_scatter is None:
        res.loc['scatter'] = 0
    else:
        res.loc['scatter'] = reg_scatter.shape[1] - 1
    res.loc['has_meme'] = False # update later
    # may also want to add the stats from enrich

    # save output
    folder = 'iModulon_files/'+str(k)+'/'
    if not(os.path.isdir(folder)):
        os.mkdir(folder)
    res.to_csv(folder+str(k)+'_meta.csv')
    gene_table.to_csv(folder+str(k)+'_gene_table.csv')
    gene_hist.to_csv(folder+str(k)+'_gene_hist.csv')
    gene_scatter.to_csv(folder+str(k)+'_gene_scatter.csv')
    act_bar.to_csv(folder+str(k)+'_activity_bar.csv')
    if not(reg_venn is None):
        reg_venn.to_csv(folder+str(k)+'_reg_venn.csv')
    if not(reg_scatter is None):
        reg_scatter.to_csv(folder+str(k)+'_reg_scatter.csv')
    ica_data.S[k].to_csv(folder+str(k)+'_gene_weights.csv')
    ica_data.A.loc[k].to_csv(folder+str(k)+'_activity.csv')

In [39]:
for k, row in enrich.iterrows():
    make_directory(ica_data, k, row, links, sample_meta)

../../../py/gene_scatter.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res.link[g] = links[g]
/home/kevin/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:795: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/home/kevin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
/home/kevin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: p

MISSING TF: GuanineRiboswitch
MISSING TF: GuanineRiboswitch


/home/kevin/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:795: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/home/kevin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
/home/kevin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
/home/kevin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:44: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to

MISSING TF: SigS
MISSING TF: SigS


../../../py/gene_scatter.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res.link[g] = links[g]
/home/kevin/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:795: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/home/kevin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
/home/kevin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: p

# Gene Files

In [30]:
# Filter out genes with no ICA data
reduced_gene_info = gene_info.copy()
reduced_gene_info = reduced_gene_info[reduced_gene_info.index.isin(ica_data.X.index)]

In [43]:
# Information for Gene Metadata

def make_gene_files(ica_data, k, row, links, sample_meta):

    # generate metadata df
    res = pd.Series(index=['gene_id', 'name', 'operon', 'product',
                           'cog', 'regulator(s)', 'link'])
    res.loc['gene_id'] = k
    res.loc['name'] = row.gene_name
    res.loc['operon'] = row.operon
    res.loc['product'] = row['product']
    res.loc['cog'] = row.cog
    res.loc['regulator(s)'] = ", ".join(trn[trn.gene_id == k].TF.to_list())
    try:
        res.loc['link'] = '<a href="' + str(links[k]) + '">'+ 'AureoWiki' + '</a>'
    except:
        res.loc['link'] = None
    res.fillna(value="<i> Not Available </i>", inplace = True) 
    
    # save output
    folder = 'gene_page_files/'+str(k)+'/'
    if not(os.path.isdir(folder)):
        os.mkdir(folder)
    res.to_csv(folder+str(k)+'_meta.csv')

In [44]:
# generate metadata csv for each gene
for k, row in reduced_gene_info.iterrows():
    make_gene_files(ica_data, k, row, links, sample_meta)

/home/katherine/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7 DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [33]:
# activity plot + expression for each gene
for gene_id in reduced_gene_info.index:
    this_fig = gene_activity_bar_df(ica_data, gene_id, sample_meta)
    folder = 'gene_page_files/'+str(gene_id)+'/'
    this_fig.to_csv(folder + gene_id + '_activity_bar.csv')
    ica_data.X.loc[gene_id].to_csv(folder+gene_id+'_expression.csv')

In [42]:
# table for each gene

# generate df with all iM genes indicated T/F 
im_genes= ica_data.S.T.copy()
for k,row in ica_data.S.T.iterrows():
    im_genes.loc[k,:] = abs(row) > abs(ica_data.thresholds[k])
    
# iM_table: start with main component of iM page
im_table = enrich[['name', 'Regulator', 'Function', 'Category']]
im_table = im_table.rename(columns={'name':'Name'})
im_table.index.name = 'k'

#loop through genes:
for gene_id in reduced_gene_info.index:
    perGene_table = im_table.copy()
    perGene_table.insert(column ='in_iM', value = im_genes.loc[:, gene_id], loc = 1)
    perGene_table.insert(column ='gene_weight', value = ica_data.S.loc[gene_id, :], loc = 2)

    #sort
    perGene_table = (perGene_table.assign(A=perGene_table['gene_weight'].abs())
            .sort_values(['in_iM','A'],ascending=[False, False])
            .drop('A', 1))
    folder = 'gene_page_files/'+str(gene_id)+'/'
    perGene_table.to_csv(folder + gene_id + '_perGene_table.csv')

### make json file for Search

In [37]:
# Get df in correct format
gene_df = reduced_gene_info.copy()
gene_df["gene_id"] = reduced_gene_info.index
gene_df = gene_df[['gene_name', "gene_id", "product"]]
gene_df = gene_df.sort_values(by="gene_name")
gene_df["gene_name"] = gene_df["gene_name"].fillna(value = "Unnamed Gene")
gene_df["product"] = gene_df["product"].fillna(value = "not available")

#create gene info json
gene_df.to_json('./gene_page_files/gene_list.json', orient="records")